# LSTM 문장생성

## 데이터준비

qus : 질문문항  
seq_len : 질문문항 길이 (총 단어 길이)  
answer : 질문에 대한 답변  
pred_clean : 질문문항이 속한 label 번호  

질문문항

0 성장과정  
1 성격 장단점, 생활신조 / 윤리의식(환경, 직업윤리)  
2 직무역량 (강점만 어필하는 느낌) / 강약점 (직무상의 강약점, 학과, 자격증, 수강과목 따라서 이런일에 역량을 발휘할 수 있습니다.)  
3 지원동기 및 입사후 포부 / 입사를 위한 준비과정 / 회사 이해도 (어떤 사업을 하고 있는지, 혁신 (입사 후 제안 서비스))  
4 사회경험 (인턴, 동아리, 공모전, 봉사활동, 아르바이트 및 학창시절, 도전, 성취 경험 )    
5 경력사항 (일을 했던 경력)  
6 사회이슈 (사회의 특정 사건에 대해서 어떤 견해를 가지고 있는지)  
7 조직적응력, 사회성 (대인관계) > 팀워크 / 소통 (이견있을 때 어떻게 해결? / 리더십)  
8 문제해결력 (의사결정-상사가 지시한 일/ 시한이 급한 일 어떤 일에 우선순위를 둘 것인가, 시나리오 문제, 정보능력)  

In [0]:
import pandas as pd
path ='/content/drive/My Drive/tabditor/소스코드/모델1. LSTM (문장생성)/ans_qus_pred_data2.csv'

train =pd.read_csv(path)

In [3]:
print(train.shape)
train

(20154, 5)


,Unnamed: 0,qus,seq_len,answer,pred_clean
0,0,"귀하가 스스로 문제를 발견하고, 이를 해결했던 경험에 대해 기술해 주세요.(문제발견...",18,"['\n올해, 기숙사 학생들의 원활한 기숙사 이용을 도모하기 위해 기숙사 자치위원을...",4
1,1,의사소통 과정에서 발생한 오해를 풀기 위해 노력을 기울였던 경험에 대해 기술해주세요...,19,['\n작년 여름 친구들과 인천부터 부산까지의 ‘자전거국토종주’를 완주한 경험이 있...,4
2,2,"우리공사 및 지원한 직무에 관심이 생긴 계기를 기술하고, 우리공사에서 근무하면서 어...",17,['\n중학교 재학 중 방학숙제로 부화기를 제작했습니다. 처음 접해보는 전기회로 제...,3
3,3,지원 분야와 관련된 우리공사의 주요 사업 중 본인이 어떤 부분에 기여하고 싶은지 기...,27,"['\n저는 항상 도전하는 인재입니다. 어려움이 찾아와도 발판 삼아 이겨내고, 목표...",2
4,4,지원 분야와 관련된 우리공사의 주요 사업 중 본인이 어떤 부분에 기여하고 싶은지 기...,27,"['\n저는 항상 도전하는 인재입니다. 어려움이 찾아와도 발판 삼아 이겨내고, 목표...",3
...,...,...,...,...,...
20149,20149,"본인이 속했던 조직(ex. 친구, 동아리, 가족, 기타모임 등)의 화합과 융화를 이...",20,['\n[`무엇인가`를 배웠습니다]\n 봉사활동을 처음 시작할 당시 봉사시간보다는 ...,7
20150,20150,자기자신을 잘 표현할 수 있는 경험과 사례를 자유롭게 기술하시오. (1000자),10,['\n[부지런한 일개미형 조사자]\n전공과목 중 ‘소비자 행동분석’에서 ‘사교육 ...,2
20151,20151,인생에서 성취하고 싶은 것과 이를 위해 어떠한 활동과 노력을 하고 있는지 기술하시오.,12,"['\n[세계 속에 한국 문화 ‘물들이기’]\n제가 인생에서 성취하고 싶은 것은, ...",2
20152,20152,샘표식품㈜에 지원하는 구체적인 이유와 샘표식품에서 여러분을 선택해야 하는 이유에 대...,13,['\n[세계가 샘내는 기업 샘표]\n68년의 전통과 최고참 상표를 유지하고 있는 ...,3


질문문항 label 4에 속한 질문만 가져오기  
(학습시키고자 하는 질문문항 label 번호로 수정가능)

In [0]:
data = train[train['pred_clean']==4]

In [8]:
print(data.shape)
data.head()

(6233, 5)


,Unnamed: 0,qus,seq_len,answer,pred_clean
0,0,"귀하가 스스로 문제를 발견하고, 이를 해결했던 경험에 대해 기술해 주세요.(문제발견...",18,"['\n올해, 기숙사 학생들의 원활한 기숙사 이용을 도모하기 위해 기숙사 자치위원을...",4
1,1,의사소통 과정에서 발생한 오해를 풀기 위해 노력을 기울였던 경험에 대해 기술해주세요...,19,['\n작년 여름 친구들과 인천부터 부산까지의 ‘자전거국토종주’를 완주한 경험이 있...,4
6,6,지금까지의 경험 중 해결하기 힘든 문제에 부딪혀 어려움을 겪었으나 이를 극복한 사례...,15,"['\n""매출 손실을 막기 위한 관리자가 지녀야 할 책임감""\n\n2년 전 카페의 ...",4
12,12,"나에게, 기업에게 정직함이란",3,['\n저에게 정직함이란 스스로 부끄럽지 않은 행동입니다.\n작년 일본으로 친구와 ...,4
13,13,지금까지 가장 큰 성공과 큰 실패,6,['\n대학 2학년 때 가상 스포츠 행사를 통한 현실적인 기업의 마케팅 방법과 예산...,4


정규표현식을 이용한 특수문자 제거

In [9]:
import re
import ast
pat = re.compile('[^0-9가-힣a-zA-Z\s]')
pat2 = re.compile('[\n]')
# data['answer'].apply(lambda x : pat.findall(x))#.replace('\n','')))
data['answer'] = data['answer'].apply(lambda x : pat.sub('', (ast.literal_eval(x)[0].replace('\n', ''))))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
data.head()

,Unnamed: 0,qus,seq_len,answer,pred_clean
0,0,"귀하가 스스로 문제를 발견하고, 이를 해결했던 경험에 대해 기술해 주세요.(문제발견...",18,올해 기숙사 학생들의 원활한 기숙사 이용을 도모하기 위해 기숙사 자치위원을 지원했습...,4
1,1,의사소통 과정에서 발생한 오해를 풀기 위해 노력을 기울였던 경험에 대해 기술해주세요...,19,작년 여름 친구들과 인천부터 부산까지의 자전거국토종주를 완주한 경험이 있습니다 체력...,4
6,6,지금까지의 경험 중 해결하기 힘든 문제에 부딪혀 어려움을 겪었으나 이를 극복한 사례...,15,매출 손실을 막기 위한 관리자가 지녀야 할 책임감2년 전 카페의 관리자로 근무하며 ...,4
12,12,"나에게, 기업에게 정직함이란",3,저에게 정직함이란 스스로 부끄럽지 않은 행동입니다작년 일본으로 친구와 여행을 갔습니...,4
13,13,지금까지 가장 큰 성공과 큰 실패,6,대학 2학년 때 가상 스포츠 행사를 통한 현실적인 기업의 마케팅 방법과 예산을 수립...,4


## Tokenize

keras tokenizer를 이용하여 tokenizing  
1000개 가량의 데이터만 사용했습니다.
  

In [12]:
from keras.preprocessing.text import Tokenizer

tknr = Tokenizer()
tknr.fit_on_texts(data['answer'][:1000])

Using TensorFlow backend.


In [14]:
vocab_size = len(tknr.word_index)+1
vocab_size

39629

sequence 데이터 준비

In [0]:
sequences = []
for line in data['answer'][:1000]:
    
    liner = tknr.texts_to_sequences([line])[0]
    
    for i in range(1,len(liner)):
        seq = liner[:i+1]
        sequences.append(seq)

In [18]:
sequences[:10]

[[1395, 1510],
 [1395, 1510, 316],
 [1395, 1510, 316, 525],
 [1395, 1510, 316, 525, 1510],
 [1395, 1510, 316, 525, 1510, 15591],
 [1395, 1510, 316, 525, 1510, 15591, 15592],
 [1395, 1510, 316, 525, 1510, 15591, 15592, 5],
 [1395, 1510, 316, 525, 1510, 15591, 15592, 5, 1510],
 [1395, 1510, 316, 525, 1510, 15591, 15592, 5, 1510, 15593],
 [1395, 1510, 316, 525, 1510, 15591, 15592, 5, 1510, 15593, 1511]]

In [19]:
maxlen = max([len(i) for i in sequences])
maxlen
# 문장길이가 가장 긴 문장의 길이는 436입니다.

436

In [0]:
# 다른 문장들도 436 길이를 맞춰주기 위해 pad를 넣어줍니다. 
from keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences,maxlen= maxlen,padding='pre')

In [21]:
sequences[:100]

array([[   0,    0,    0, ...,    0, 1395, 1510],
       [   0,    0,    0, ..., 1395, 1510,  316],
       [   0,    0,    0, ..., 1510,  316,  525],
       ...,
       [   0,    0,    0, ...,  220,  628, 9513],
       [   0,    0,    0, ...,  628, 9513,  414],
       [   0,    0,    0, ..., 9513,  414, 6846]], dtype=int32)

x train과 y train 준비  
문장생성 task는 다음 단어가 y train이 되기 때문에 현재 문장에서 다음 단어를 split하여 y train을 만들어 줍니다. 

In [0]:
x_train = sequences[:,:-1]
y_train = sequences[:,-1]

In [30]:
max_y = max([i for i in y_train])
max_y

39628

In [36]:
y_train[:10]

array([ 1510,   316,   525,  1510, 15591, 15592,     5,  1510, 15593,
        1511], dtype=int32)

## 모델 학습


In [37]:
from keras.layers import Dense,Embedding,LSTM
from keras.models import Sequential

model = Sequential()
model.add(Embedding(max_y+1,10,input_length=maxlen-1))
model.add(LSTM(64))
model.add(Dense(max_y+1,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics=['accuracy']) #metrics = 평가척도


In [47]:
model.fit(x_train,y_train,epochs=10,verbose=2)

Epoch 1/10


KeyboardInterrupt: ignored

In [0]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=maxlen-1, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
        # l_pred_probvec = model.predict(encoded)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [0]:
#문장 자르기 위한 전처리 함수
import re
def reg(strs):
    pat = re.compile('니다')
    idx_end = pat.search(strs).end()
    return strs[:idx_end]

In [46]:
print(reg(sentence_generation(model, tknr, '제가', 15)))
print(reg(sentence_generation(model, tknr, '대학원', 15)))
print(reg(sentence_generation(model, tknr, '영업', 15)))
print(reg(sentence_generation(model, tknr, '경험', 15)))
print(reg(sentence_generation(model, tknr, '그리고', 15)))
print(reg(sentence_generation(model, tknr, '은행', 15)))
print(reg(sentence_generation(model, tknr, '하지만', 15)))
print(reg(sentence_generation(model, tknr, '생각하며', 15)))

AttributeError: ignored